# CNN + 분류 모델 

In [1]:
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

In [2]:
import os
import numpy as np
import pandas as pd
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm

# 데이터 준비 및 전처리
- combined_result.csv (raw_data + 기타소음4)

In [3]:
#학습 데이터 준비
df = pd.read_csv("combined_result.csv", encoding='utf-8')
feature_cols = [f'mfcc_{i}' for i in range(1, 51)]
df = df[feature_cols + ['category_03']]

In [4]:
df.shape

(18021, 51)

In [5]:
df.info

<bound method DataFrame.info of            mfcc_1      mfcc_2     mfcc_3     mfcc_4     mfcc_5     mfcc_6  \
0     -391.004300  107.641975  64.204640  33.718952  20.630340  13.337987   
1     -233.662840   87.917620  45.434906  19.435001   9.098841  32.400368   
2     -318.161530  126.123180  43.729805 -11.429483  -5.223235  13.667998   
3     -385.209260  140.265410  57.263832  -7.932145  -3.877982  14.155136   
4     -382.124500  150.004900  61.439040 -17.890905  -7.740589  15.947204   
...           ...         ...        ...        ...        ...        ...   
18016 -312.639191  162.565536 -36.581287   7.150859 -18.033703  -0.001340   
18017 -321.451477  208.936478 -35.191029 -10.881365 -13.412726   2.397377   
18018 -428.432526  152.495697   4.484459  36.305748 -22.177618  14.208892   
18019 -240.417511  206.323395 -62.882484 -26.428501 -30.405287  -5.296096   
18020 -301.803223  225.951523 -34.041290 -76.669899 -21.702627   5.007879   

          mfcc_7     mfcc_8     mfcc_9    m

In [6]:
df['category_03'].value_counts()

이륜차주행음    4735
이륜차경적     4560
차량경적      3189
차량사이렌     1990
차량주행음     1682
헬리콥터       435
기차         281
고양이        274
지하철        245
개          217
비행기        191
세탁기        120
청소기        102
Name: category_03, dtype: int64

# 소음 카테고리 변환

In [7]:
# 소음 카테고리 변환 함수
def categorize_noise(category):
    if category in ['이륜차경적']:
        return '이륜차경적'
    elif category in ['이륜차주행음']:
        return '이륜차주행음'
    elif category in ['차량사이렌']:
        return '차량사이렌'
    elif category in ['차량주행음']:
        return '차량주행음'
    elif category in ['차량경적']:
        return '차량경적'
    else:
        return '기타소음'

df['label'] = df['category_03'].apply(categorize_noise)
df = df.drop('category_03', axis=1)

X = df[feature_cols].values  # MFCC 특징 값
y = df['label'].values       # 레이블

# 레이블 인코딩 (문자 → 숫자)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)  # One-hot 인코딩

# 데이터셋 분할 (훈련:테스트 = 80:20)
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42, stratify=y)

# CNN 입력 형태로 변환 (3D 텐서: 샘플 수 x 시간 축 x 특징 수)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [8]:
df['label'].value_counts()

이륜차주행음    4735
이륜차경적     4560
차량경적      3189
차량사이렌     1990
기타소음      1865
차량주행음     1682
Name: label, dtype: int64

# 모델 생성

## CNN

In [9]:
# CNN 모델 정의
model = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),
    
    Conv1D(128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(y_categorical.shape[1], activation='softmax')  # 출력층 (카테고리 수만큼 출력)
])

In [10]:
# 모델 컴파일 및 학습
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test))

# 테스트 정확도 출력
test_loss, test_accuracy = model.evaluate(X_test, y_test)

# 테스트 데이터에 대한 예측 수행
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # 예측된 클래스 인덱스
y_true_classes = np.argmax(y_test, axis=1)  # 실제 클래스 인덱스

# 정확도 계산
accuracy = accuracy_score(y_true_classes, y_pred_classes)
print(f"Accuracy: {accuracy:.4f}")

# 분류 보고서 출력
class_names = label_encoder.classes_
report = classification_report(y_true_classes, y_pred_classes, target_names=class_names)
print("Classification Report:\n", report)

# 혼동 행렬 출력
conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)
print("Confusion Matrix:\n", conf_matrix)

Epoch 1/30
451/451 [==============================] - 3s 6ms/step - loss: 0.8353 - accuracy: 0.7090 - val_loss: 0.4207 - val_accuracy: 0.8480
Epoch 2/30
451/451 [==============================] - 3s 6ms/step - loss: 0.4634 - accuracy: 0.8246 - val_loss: 0.3068 - val_accuracy: 0.8946
Epoch 3/30
451/451 [==============================] - 3s 6ms/step - loss: 0.3797 - accuracy: 0.8584 - val_loss: 0.2581 - val_accuracy: 0.9079
Epoch 4/30
451/451 [==============================] - 2s 5ms/step - loss: 0.3400 - accuracy: 0.8744 - val_loss: 0.2460 - val_accuracy: 0.9065
Epoch 5/30
451/451 [==============================] - 2s 5ms/step - loss: 0.3126 - accuracy: 0.8885 - val_loss: 0.2271 - val_accuracy: 0.9168
Epoch 6/30
451/451 [==============================] - 2s 5ms/step - loss: 0.2877 - accuracy: 0.8982 - val_loss: 0.2178 - val_accuracy: 0.9265
Epoch 7/30
451/451 [==============================] - 2s 5ms/step - loss: 0.2680 - accuracy: 0.9012 - val_loss: 0.2047 - val_accuracy: 0.9212
Epoch 

In [12]:
# 모델 저장
model.save('cnn_model_6classfication.h5')
print("모델이 cnn_model_6classfication.h5 파일로 저장되었습니다.")

모델이 cnn_model_6classfication.h5 파일로 저장되었습니다.


### 성능 개선
- 배치 정규화 (Batch Normalisation) 레이어 Conv1D 와 Dense 추가
- 앙상블 기법 적용
- ReduceROnPlateau 콜백 사용 
- 모델 구조 함수화

In [21]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Dropout, Flatten, Input, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 데이터 준비 (기존 코드와 동일)
df = pd.read_csv("combined_result.csv", encoding='utf-8')
feature_cols = [f'mfcc_{i}' for i in range(1, 51)]
df = df[feature_cols + ['category_03']]

def categorize_noise(category):
    if category in ['이륜차경적']:
        return '이륜차경적'
    elif category in ['이륜차주행음']:
        return '이륜차주행음'
    elif category in ['차량사이렌']:
        return '차량사이렌'
    elif category in ['차량주행음']:
        return '차량주행음'
    elif category in ['차량경적']:
        return '차량경적'
    else:
        return '기타소음'

df['label'] = df['category_03'].apply(categorize_noise)
df = df.drop('category_03', axis=1)

X = df[feature_cols].values
y = df['label'].values

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42, stratify=y)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# 개선된 CNN 모델 정의
def create_model():
    inputs = Input(shape=(X_train.shape[1], 1))
    x = Conv1D(64, kernel_size=3, activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Dropout(0.3)(x)
    
    x = Conv1D(128, kernel_size=3, activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Dropout(0.3)(x)
    
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    outputs = Dense(y_categorical.shape[1], activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

# 앙상블 모델 생성
def create_ensemble(num_models=3):
    models = []
    for _ in range(num_models):
        model = create_model()
        models.append(model)
    return models

# 앙상블 모델 학습
ensemble = create_ensemble()

# 학습률 스케줄링
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

# 모델 학습
histories = []
for i, model in enumerate(ensemble):
    print(f"Training model {i+1}/{len(ensemble)}")
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test), callbacks=[reduce_lr])
    histories.append(history)

# 앙상블 예측
def ensemble_predict(models, X):
    predictions = [model.predict(X) for model in models]
    return np.mean(predictions, axis=0)

# 테스트 데이터에 대한 앙상블 예측
y_pred_ensemble = ensemble_predict(ensemble, X_test)
y_pred_classes = np.argmax(y_pred_ensemble, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# 정확도 계산
accuracy = accuracy_score(y_true_classes, y_pred_classes)
print(f"Ensemble Accuracy: {accuracy:.4f}")

# 분류 보고서 출력
class_names = label_encoder.classes_
report = classification_report(y_true_classes, y_pred_classes, target_names=class_names)
print("Classification Report:\n", report)

# 혼동 행렬 출력
conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)
print("Confusion Matrix:\n", conf_matrix)


Training model 1/3
Epoch 1/30
451/451 [==============================] - 5s 10ms/step - loss: 0.6533 - accuracy: 0.7648 - val_loss: 0.3348 - val_accuracy: 0.8810
Epoch 2/30
451/451 [==============================] - 4s 10ms/step - loss: 0.4194 - accuracy: 0.8479 - val_loss: 0.2756 - val_accuracy: 0.9021
Epoch 3/30
451/451 [==============================] - 5s 11ms/step - loss: 0.3585 - accuracy: 0.8683 - val_loss: 0.2494 - val_accuracy: 0.9054
Epoch 4/30
451/451 [==============================] - 4s 9ms/step - loss: 0.3247 - accuracy: 0.8846 - val_loss: 0.2151 - val_accuracy: 0.9232
Epoch 5/30
451/451 [==============================] - 4s 9ms/step - loss: 0.2960 - accuracy: 0.8931 - val_loss: 0.2529 - val_accuracy: 0.9090
Epoch 6/30
451/451 [==============================] - 4s 9ms/step - loss: 0.2791 - accuracy: 0.8998 - val_loss: 0.2096 - val_accuracy: 0.9234
Epoch 7/30
451/451 [==============================] - 4s 9ms/step - loss: 0.2646 - accuracy: 0.9038 - val_loss: 0.2070 - val_a

In [22]:
# 모델 저장
model.save('updated_cnn_model_6classfication.h5')
print("모델이 updated_cnn_model_6classfication.h5 파일로 저장되었습니다.")

모델이 updated_cnn_model_6classfication.h5 파일로 저장되었습니다.


In [24]:
from tensorflow import keras
import numpy as np

# 저장된 모델 로드
loaded_model = keras.models.load_model('updated_cnn_model_6classfication.h5')

# 모델 구조 확인
loaded_model.summary()

# 테스트 데이터로 예측 수행
y_pred = loaded_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# 정확도 계산
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_true_classes, y_pred_classes)
print(f"Loaded Model Accuracy: {accuracy:.4f}")

# 분류 보고서 출력
from sklearn.metrics import classification_report
class_names = label_encoder.classes_
report = classification_report(y_true_classes, y_pred_classes, target_names=class_names)
print("Classification Report:\n", report)


Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 50, 1)]           0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 48, 64)            256       
_________________________________________________________________
batch_normalization_6 (Batch (None, 48, 64)            256       
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 24, 64)            0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 24, 64)            0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 22, 128)           24704     
_________________________________________________________________
batch_normalization_7 (Batch (None, 22, 128)           512 

### 차량 주행음 개선

In [27]:
from sklearn.utils import class_weight
from tensorflow.keras.optimizers import Adam

# 클래스 가중치 계산
class_weights = class_weight.compute_class_weight('balanced',
                                                 classes=np.unique(np.argmax(y_train, axis=1)),
                                                 y=np.argmax(y_train, axis=1))
class_weight_dict = dict(enumerate(class_weights))

# 모델 컴파일 및 학습
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test), class_weight=class_weight_dict, callbacks=[reduce_lr])


Epoch 1/30
451/451 [==============================] - 7s 12ms/step - loss: 0.1611 - accuracy: 0.9391 - val_loss: 0.1729 - val_accuracy: 0.9393
Epoch 2/30
451/451 [==============================] - 4s 8ms/step - loss: 0.1671 - accuracy: 0.9381 - val_loss: 0.1582 - val_accuracy: 0.9448
Epoch 3/30
451/451 [==============================] - 4s 8ms/step - loss: 0.1649 - accuracy: 0.9368 - val_loss: 0.1851 - val_accuracy: 0.9293
Epoch 4/30
451/451 [==============================] - 4s 8ms/step - loss: 0.1575 - accuracy: 0.9398 - val_loss: 0.1573 - val_accuracy: 0.9437
Epoch 5/30
451/451 [==============================] - 4s 8ms/step - loss: 0.1606 - accuracy: 0.9387 - val_loss: 0.1652 - val_accuracy: 0.9393
Epoch 6/30
451/451 [==============================] - 4s 8ms/step - loss: 0.1554 - accuracy: 0.9407 - val_loss: 0.1743 - val_accuracy: 0.9334
Epoch 7/30
451/451 [==============================] - 4s 8ms/step - loss: 0.1499 - accuracy: 0.9422 - val_loss: 0.1791 - val_accuracy: 0.9318
Epoch

In [30]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Dropout, Flatten, Input, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.utils import class_weight

# 데이터 준비 (기존 코드와 동일)
df = pd.read_csv("combined_result.csv", encoding='utf-8')
feature_cols = [f'mfcc_{i}' for i in range(1, 51)]
df = df[feature_cols + ['category_03']]

def categorize_noise(category):
    if category in ['이륜차경적']:
        return '이륜차경적'
    elif category in ['이륜차주행음']:
        return '이륜차주행음'
    elif category in ['차량사이렌']:
        return '차량사이렌'
    elif category in ['차량주행음']:
        return '차량주행음'
    elif category in ['차량경적']:
        return '차량경적'
    else:
        return '기타소음'

df['label'] = df['category_03'].apply(categorize_noise)
df = df.drop('category_03', axis=1)

X = df[feature_cols].values
y = df['label'].values

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42, stratify=y)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# 클래스 가중치 계산
class_weights = class_weight.compute_class_weight('balanced',
                                                 classes=np.unique(np.argmax(y_train, axis=1)),
                                                 y=np.argmax(y_train, axis=1))
class_weight_dict = dict(enumerate(class_weights))

# 개선된 CNN 모델 정의
def create_model():
    inputs = Input(shape=(X_train.shape[1], 1))
    x = Conv1D(64, kernel_size=3, activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Dropout(0.3)(x)
    
    x = Conv1D(128, kernel_size=3, activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Dropout(0.3)(x)
    
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    outputs = Dense(y_categorical.shape[1], activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

# 앙상블 모델 생성
def create_ensemble(num_models=3):
    models = []
    for _ in range(num_models):
        model = create_model()
        models.append(model)
    return models

# 앙상블 모델 학습
ensemble = create_ensemble()

# 학습률 스케줄링
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

# 모델 학습
histories = []
for i, model in enumerate(ensemble):
    print(f"Training model {i+1}/{len(ensemble)}")
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test), 
                        class_weight=class_weight_dict, callbacks=[reduce_lr])
    histories.append(history)

# 앙상블 예측
def ensemble_predict(models, X):
    predictions = [model.predict(X) for model in models]
    return np.mean(predictions, axis=0)

# 테스트 데이터에 대한 앙상블 예측
y_pred_ensemble = ensemble_predict(ensemble, X_test)
y_pred_classes = np.argmax(y_pred_ensemble, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# 정확도 계산
accuracy = accuracy_score(y_true_classes, y_pred_classes)
print(f"Ensemble Accuracy: {accuracy:.4f}")

# 분류 보고서 출력
class_names = label_encoder.classes_
report = classification_report(y_true_classes, y_pred_classes, target_names=class_names)
print("Classification Report:\n", report)

# 혼동 행렬 출력
conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)
print("Confusion Matrix:\n", conf_matrix)


Training model 1/3
Epoch 1/30
451/451 [==============================] - 9s 16ms/step - loss: 0.7234 - accuracy: 0.7426 - val_loss: 0.3486 - val_accuracy: 0.8741
Epoch 2/30
451/451 [==============================] - 7s 15ms/step - loss: 0.4700 - accuracy: 0.8328 - val_loss: 0.2820 - val_accuracy: 0.8968
Epoch 3/30
451/451 [==============================] - 8s 17ms/step - loss: 0.3877 - accuracy: 0.8623 - val_loss: 0.2684 - val_accuracy: 0.9024
Epoch 4/30
451/451 [==============================] - 7s 16ms/step - loss: 0.3487 - accuracy: 0.8746 - val_loss: 0.2698 - val_accuracy: 0.9021
Epoch 5/30
451/451 [==============================] - 7s 16ms/step - loss: 0.3232 - accuracy: 0.8810 - val_loss: 0.2452 - val_accuracy: 0.9101
Epoch 6/30
451/451 [==============================] - 7s 16ms/step - loss: 0.3095 - accuracy: 0.8880 - val_loss: 0.2746 - val_accuracy: 0.8999
Epoch 7/30
451/451 [==============================] - 8s 17ms/step - loss: 0.2987 - accuracy: 0.8914 - val_loss: 0.2600 - v

In [31]:
# 모델 저장
model.save('updated2_cnn_model_6classfication.h5')
print("모델이 updated2_cnn_model_6classfication.h5 파일로 저장되었습니다.")

모델이 updated2_cnn_model_6classfication.h5 파일로 저장되었습니다.


### 차량주행음 - Precision 과 f1-score 개선
- 오버샘플링

In [35]:
from imblearn.over_sampling import RandomOverSampler
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Dropout, Flatten, Input, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 데이터 준비
df = pd.read_csv("combined_result.csv", encoding='utf-8')
feature_cols = [f'mfcc_{i}' for i in range(1, 51)]
df = df[feature_cols + ['category_03']]

def categorize_noise(category):
    if category in ['이륜차경적']:
        return '이륜차경적'
    elif category in ['이륜차주행음']:
        return '이륜차주행음'
    elif category in ['차량사이렌']:
        return '차량사이렌'
    elif category in ['차량주행음']:
        return '차량주행음'
    elif category in ['차량경적']:
        return '차량경적'
    else:
        return '기타소음'

df['label'] = df['category_03'].apply(categorize_noise)
df = df.drop('category_03', axis=1)

X = df[feature_cols].values
y = df['label'].values

# 레이블 인코딩 및 데이터 분할
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y)

# 오버샘플링 적용 (훈련 데이터만)
ros = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

# 원-핫 인코딩
y_train_categorical = to_categorical(y_train_resampled)
y_test_categorical = to_categorical(y_test)

# 데이터 형태 변환
X_train_resampled = X_train_resampled.reshape(X_train_resampled.shape[0], X_train_resampled.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# CNN 모델 정의
def create_model():
    inputs = Input(shape=(X_train_resampled.shape[1], 1))
    x = Conv1D(64, kernel_size=3, activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Dropout(0.3)(x)
    
    x = Conv1D(128, kernel_size=3, activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Dropout(0.3)(x)
    
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    outputs = Dense(y_test_categorical.shape[1], activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

# 앙상블 모델 생성
def create_ensemble(num_models=3):
    models = []
    for _ in range(num_models):
        model = create_model()
        models.append(model)
    return models

# 앙상블 모델 학습
ensemble = create_ensemble()

# 학습률 스케줄링 콜백 정의
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

# 모델 학습 (오버샘플링된 데이터 사용)
histories = []
for i, model in enumerate(ensemble):
    print(f"Training model {i+1}/{len(ensemble)}")
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train_resampled, y_train_categorical,
                        epochs=30,
                        batch_size=32,
                        validation_data=(X_test, y_test_categorical),
                        callbacks=[reduce_lr])
    histories.append(history)

# 앙상블 예측 함수 정의
def ensemble_predict(models, X):
    predictions = [model.predict(X) for model in models]
    return np.mean(predictions, axis=0)

# 테스트 데이터에 대한 앙상블 예측 수행
y_pred_ensemble = ensemble_predict(ensemble, X_test)

# 클래스별로 가장 높은 확률을 가진


Training model 1/3
Epoch 1/30
711/711 [==============================] - 7s 9ms/step - loss: 0.6138 - accuracy: 0.7742 - val_loss: 0.3225 - val_accuracy: 0.8832
Epoch 2/30
711/711 [==============================] - 6s 9ms/step - loss: 0.3838 - accuracy: 0.8614 - val_loss: 0.2614 - val_accuracy: 0.9065
Epoch 3/30
711/711 [==============================] - 7s 9ms/step - loss: 0.3116 - accuracy: 0.8887 - val_loss: 0.2307 - val_accuracy: 0.9154
Epoch 4/30
711/711 [==============================] - 10s 15ms/step - loss: 0.2786 - accuracy: 0.8999 - val_loss: 0.2037 - val_accuracy: 0.9257
Epoch 5/30
711/711 [==============================] - 10s 14ms/step - loss: 0.2537 - accuracy: 0.9093 - val_loss: 0.1748 - val_accuracy: 0.9376
Epoch 6/30
711/711 [==============================] - 10s 15ms/step - loss: 0.2321 - accuracy: 0.9179 - val_loss: 0.2162 - val_accuracy: 0.9196
Epoch 7/30
711/711 [==============================] - 10s 15ms/step - loss: 0.2161 - accuracy: 0.9229 - val_loss: 0.2102 - 

In [36]:
# 클래스별로 가장 높은 확률을 가진 클래스 인덱스를 선택
y_pred_classes = np.argmax(y_pred_ensemble, axis=1)
y_true_classes = np.argmax(y_test_categorical, axis=1)

# 정확도 계산 및 출력
accuracy = accuracy_score(y_true_classes, y_pred_classes)
print(f"Ensemble Accuracy: {accuracy:.4f}")

# 분류 보고서 출력
class_names = label_encoder.classes_
report = classification_report(y_true_classes, y_pred_classes, target_names=class_names)
print("Classification Report:\n", report)

# 혼동 행렬 출력
conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)
print("Confusion Matrix:\n", conf_matrix)

Ensemble Accuracy: 0.9481
Classification Report:
               precision    recall  f1-score   support

        기타소음       0.96      0.97      0.97       373
       이륜차경적       0.98      0.95      0.97       912
      이륜차주행음       0.96      0.92      0.94       947
        차량경적       0.94      0.96      0.95       638
       차량사이렌       0.98      0.96      0.97       398
       차량주행음       0.81      0.96      0.88       337

    accuracy                           0.95      3605
   macro avg       0.94      0.95      0.95      3605
weighted avg       0.95      0.95      0.95      3605

Confusion Matrix:
 [[362   1   6   0   2   2]
 [  0 869   2  41   0   0]
 [ 11   0 871   0   2  63]
 [  0  14  10 610   1   3]
 [  2   0   3   1 383   9]
 [  1   0  11   0   2 323]]


## LightGBM

In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 데이터 준비
df = pd.read_csv("combined_result.csv", encoding='utf-8')
feature_cols = [f'mfcc_{i}' for i in range(1, 51)]
df = df[feature_cols + ['category_03']]

# 소음 카테고리 변환 함수
def categorize_noise(category):
    if category in ['이륜차경적']:
        return '이륜차경적'
    elif category in ['이륜차주행음']:
        return '이륜차주행음'
    elif category in ['차량사이렌']:
        return '차량사이렌'
    elif category in ['차량주행음']:
        return '차량주행음'
    elif category in ['차량경적']:
        return '차량경적'
    else:
        return '기타소음'

df['label'] = df['category_03'].apply(categorize_noise)
df = df.drop('category_03', axis=1)

X = df[feature_cols].values  # MFCC 특징 값
y = df['label'].values       # 레이블

# 레이블 인코딩 (문자 → 숫자)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# 데이터셋 분할 (훈련:테스트 = 80:20)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

# LightGBM 모델 정의 및 학습
model = LGBMClassifier(n_estimators=100, learning_rate=0.1, max_depth=-1, random_state=42)
model.fit(X_train, y_train)

# 테스트 데이터에 대한 예측 수행
y_pred = model.predict(X_test)

# 정확도 계산
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# 분류 보고서 출력
class_names = label_encoder.classes_
report = classification_report(y_test, y_pred, target_names=class_names)
print("Classification Report:\n", report)

# 혼동 행렬 출력
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)



# # 📌 5. LGBM 모델 정의 및 학습
# model = LGBMClassifier(n_estimators=100, learning_rate=0.1, max_depth=-1, random_state=42)
# model.fit(X_train, y_train)

# # 📌 6. 예측 수행
# y_pred = model.predict(X_test)

# # 📌 7. 평가 결과 출력
# acc = accuracy_score(y_test, y_pred)
# print(f"✅ LightGBM 정확도: {acc:.4f}")

# # 📌 8. 혼동 행렬 출력
# print("\n🎯 Confusion Matrix:")
# conf_matrix = pd.DataFrame(confusion_matrix(y_test, y_pred), columns=class_names)
# print(conf_matrix)

# # 📌 9. 분류 보고서 출력
# print("\n📊 Classification Report:")
# print(classification_report(y_test, y_pred, target_names=class_names.tolist()))

✅ LightGBM 정확도: 0.9451

🎯 Confusion Matrix:
   기타소음  이륜차경적  이륜차주행음  차량경적  차량사이렌  차량주행음
0   346      4      15     1      4      3
1     3    870       2    37      0      0
2    15      1     912     1      0     18
3     1     22      14   595      1      5
4     2      0       4     1    385      6
5     6      0      31     1      0    299

📊 Classification Report:
              precision    recall  f1-score   support

        기타소음       0.93      0.93      0.93       373
       이륜차경적       0.97      0.95      0.96       912
      이륜차주행음       0.93      0.96      0.95       947
        차량경적       0.94      0.93      0.93       638
       차량사이렌       0.99      0.97      0.98       398
       차량주행음       0.90      0.89      0.90       337

    accuracy                           0.95      3605
   macro avg       0.94      0.94      0.94      3605
weighted avg       0.95      0.95      0.95      3605



## ResNet

In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Activation, Add, GlobalAveragePooling1D, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 데이터 준비
df = pd.read_csv("combined_result.csv", encoding='utf-8')
feature_cols = [f'mfcc_{i}' for i in range(1, 51)]
df = df[feature_cols + ['category_03']]

# 소음 카테고리 변환 함수
def categorize_noise(category):
    if category in ['이륜차경적']:
        return '이륜차경적'
    elif category in ['이륜차주행음']:
        return '이륜차주행음'
    elif category in ['차량사이렌']:
        return '차량사이렌'
    elif category in ['차량주행음']:
        return '차량주행음'
    elif category in ['차량경적']:
        return '차량경적'
    else:
        return '기타소음'

df['label'] = df['category_03'].apply(categorize_noise)
df = df.drop('category_03', axis=1)

X = df[feature_cols].values  # MFCC 특징 값
y = df['label'].values       # 레이블

# 레이블 인코딩 (문자 → 숫자)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)  # One-hot 인코딩

# 데이터셋 분할 (훈련:테스트 = 80:20)
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42, stratify=y)

# CNN 입력 형태로 변환 (3D 텐서: 샘플 수 x 시간 축 x 특징 수)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# ResNet 블록 정의
def residual_block(x, filters, kernel_size=3, stride=1):
    shortcut = x
    x = Conv1D(filters, kernel_size, strides=stride, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(filters, kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    
    if stride != 1 or shortcut.shape[-1] != filters:
        shortcut = Conv1D(filters, 1, strides=stride, padding='same')(shortcut)
        shortcut = BatchNormalization()(shortcut)
    
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

# ResNet 모델 정의
def build_resnet(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    x = Conv1D(64, 7, strides=2, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = residual_block(x, 64)
    x = residual_block(x, 64)
    x = residual_block(x, 128, stride=2)
    x = residual_block(x, 128)
    x = residual_block(x, 256, stride=2)
    x = residual_block(x, 256)
    
    x = GlobalAveragePooling1D()(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs, outputs)
    return model

# 모델 생성 및 컴파일
model = build_resnet(input_shape=(X_train.shape[1], 1), num_classes=y_categorical.shape[1])
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test))

# 테스트 데이터에 대한 예측 수행
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # 예측된 클래스 인덱스
y_true_classes = np.argmax(y_test, axis=1)  # 실제 클래스 인덱스

# 정확도 계산
accuracy = accuracy_score(y_true_classes, y_pred_classes)
print(f"Accuracy: {accuracy:.4f}")

# 분류 보고서 출력
class_names = label_encoder.classes_
report = classification_report(y_true_classes, y_pred_classes, target_names=class_names)
print("Classification Report:\n", report)

# 혼동 행렬 출력
conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)
print("Confusion Matrix:\n", conf_matrix)


Epoch 1/30
451/451 [==============================] - 17s 31ms/step - loss: 0.4224 - accuracy: 0.8466 - val_loss: 0.4001 - val_accuracy: 0.8749
Epoch 2/30
451/451 [==============================] - 14s 30ms/step - loss: 0.2593 - accuracy: 0.9084 - val_loss: 0.2376 - val_accuracy: 0.9204
Epoch 3/30
451/451 [==============================] - 14s 31ms/step - loss: 0.1922 - accuracy: 0.9308 - val_loss: 0.1670 - val_accuracy: 0.9387
Epoch 4/30
451/451 [==============================] - 14s 31ms/step - loss: 0.1707 - accuracy: 0.9372 - val_loss: 0.2102 - val_accuracy: 0.9243
Epoch 5/30
451/451 [==============================] - 14s 31ms/step - loss: 0.1465 - accuracy: 0.9467 - val_loss: 0.2328 - val_accuracy: 0.9209
Epoch 6/30
451/451 [==============================] - 19s 42ms/step - loss: 0.1316 - accuracy: 0.9521 - val_loss: 0.2120 - val_accuracy: 0.9345
Epoch 7/30
451/451 [==============================] - 21s 46ms/step - loss: 0.1309 - accuracy: 0.9502 - val_loss: 0.1763 - val_accuracy:

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, GlobalAveragePooling1D, Dense, BatchNormalization, Add, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils import class_weight
from sklearn.preprocessing import LabelEncoder

# 데이터 준비
df = pd.read_csv("combined_result.csv", encoding='utf-8')
feature_cols = [f'mfcc_{i}' for i in range(1, 51)]
df = df[feature_cols + ['category_03']]

# 소음 카테고리 변환 함수
def categorize_noise(category):
    if category in ['이륜차경적']:
        return '이륜차경적'
    elif category in ['이륜차주행음']:
        return '이륜차주행음'
    elif category in ['차량사이렌']:
        return '차량사이렌'
    elif category in ['차량주행음']:
        return '차량주행음'
    elif category in ['차량경적']:
        return '차량경적'
    else:
        return '기타소음'

df['label'] = df['category_03'].apply(categorize_noise)
df = df.drop('category_03', axis=1)

# 특성과 레이블 분리
X = df[feature_cols].values
y = df['label'].values

# 레이블 인코딩
le = LabelEncoder()
y = le.fit_transform(y)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 데이터 증강 함수
def augment_data(X, y):
    X_augmented = []
    y_augmented = []
    
    for i in range(len(X)):
        X_augmented.append(X[i])
        y_augmented.append(y[i])
        
        # 노이즈 추가
        noise = np.random.normal(0, 0.01, X[i].shape)
        X_augmented.append(X[i] + noise)
        y_augmented.append(y[i])
        
        # 시간 축 이동
        shift = np.random.randint(-5, 6)
        X_augmented.append(np.roll(X[i], shift))
        y_augmented.append(y[i])
    
    return np.array(X_augmented), np.array(y_augmented)

# 데이터 증강 적용
X_train_augmented, y_train_augmented = augment_data(X_train, y_train)

# 데이터 형태 변환 (ResNet 입력 형태)
X_train_augmented = X_train_augmented.reshape((X_train_augmented.shape[0], X_train_augmented.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# One-hot encoding
num_classes = len(np.unique(y))
y_train_augmented = to_categorical(y_train_augmented, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)

# ResNet 모델 구축
def residual_block(x, filters, kernel_size=3, stride=1):
    shortcut = x
    x = Conv1D(filters, kernel_size, strides=stride, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv1D(filters, kernel_size, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    if shortcut.shape[-1] != filters:
        shortcut = Conv1D(filters, 1, strides=stride, padding='same')(shortcut)
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

def build_resnet(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    x = Conv1D(64, 7, strides=2, padding='same', activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = residual_block(x, 64)
    x = residual_block(x, 128, stride=2)
    x = residual_block(x, 256, stride=2)
    x = residual_block(x, 512, stride=2)
    x = GlobalAveragePooling1D()(x)
    x = Dense(num_classes, activation='softmax')(x)
    return Model(inputs, x)

# 모델 생성 및 컴파일
resnet_model = build_resnet(X_train_augmented.shape[1:], num_classes=num_classes)
optimizer = Adam(learning_rate=0.001)
resnet_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=0.00001)

# 클래스 가중치 계산
unique_classes = np.unique(y_train)
class_weights = class_weight.compute_class_weight('balanced', 
                                                  classes=unique_classes, 
                                                  y=y_train)
class_weights_dict = dict(zip(unique_classes, class_weights))

history = resnet_model.fit(X_train_augmented, y_train_augmented, epochs=30, batch_size=32, 
                           validation_data=(X_test, y_test), callbacks=[early_stopping, reduce_lr], 
                           class_weight=class_weights_dict)

# 모델 평가
test_loss, test_accuracy = resnet_model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy:.4f}")

# 클래스별 정확도 출력
y_pred = resnet_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

for i, class_name in enumerate(le.classes_):
    class_accuracy = np.mean((y_pred_classes == i) == (y_true_classes == i))
    print(f"Accuracy for class '{class_name}': {class_accuracy:.4f}")


Epoch 1/30
1352/1352 [==============================] - 93s 67ms/step - loss: 0.5031 - accuracy: 0.8219 - val_loss: 0.2931 - val_accuracy: 0.9043
Epoch 2/30
1352/1352 [==============================] - 88s 65ms/step - loss: 0.2956 - accuracy: 0.8900 - val_loss: 0.2821 - val_accuracy: 0.9004
Epoch 3/30
1352/1352 [==============================] - 87s 64ms/step - loss: 0.2332 - accuracy: 0.9119 - val_loss: 0.1842 - val_accuracy: 0.9312
Epoch 4/30
1352/1352 [==============================] - 82s 61ms/step - loss: 0.1939 - accuracy: 0.9253 - val_loss: 0.2205 - val_accuracy: 0.9234
Epoch 5/30
1352/1352 [==============================] - 81s 60ms/step - loss: 0.1646 - accuracy: 0.9359 - val_loss: 0.2177 - val_accuracy: 0.9221
Epoch 6/30
1352/1352 [==============================] - 81s 60ms/step - loss: 0.1413 - accuracy: 0.9435 - val_loss: 0.2011 - val_accuracy: 0.9334
Epoch 7/30
1352/1352 [==============================] - 84s 62ms/step - loss: 0.1164 - accuracy: 0.9520 - val_loss: 0.1944 -

# 새로운 wav 파일 분류 함수

In [10]:
def classify_audio(audio_path):
    try:
        # 음원 파일 로드
        y, sr = librosa.load(audio_path, sr=44100)

        # MFCC 특징 추출
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=50)

        # 추출된 MFCC 특징의 평균값을 사용 (각 MFCC 차수별로 하나의 값)
        mfccs_processed = np.mean(mfccs, axis=1)

        # CNN 입력 형태로 변환 (3D 텐서)
        new_data = mfccs_processed.reshape(1, -1, 1)

        # 예측 수행
        predicted_label_encoded = np.argmax(model.predict(new_data), axis=-1)[0]
        predicted_label = label_encoder.inverse_transform([predicted_label_encoded])[0]

        return predicted_label

    except Exception as e:
        print(f"Error processing audio file: {e}")
        return "Unknown"


## 기타소음

### 고양이

In [24]:
folder_path = "/home/ubuntu/data/etc_noise_data_test_04/동물/15.고양이"
results = []

for filename in tqdm(os.listdir(folder_path)):
    if filename.endswith(".wav"):
        file_path = os.path.join(folder_path, filename)
        predicted_label = classify_audio(file_path)
        results.append({"File": filename, "Predicted Label": predicted_label})

# 5. 결과를 DataFrame으로 변환하고 CSV로 저장
results_df_cat = pd.DataFrame(results)
results_df_cat.to_csv("cat.csv", index=False)
print("Classification completed. Results saved to 'cat.csv'")

100%|██████████| 274/274 [00:33<00:00,  8.15it/s]

Classification completed. Results saved to 'cat.csv'


In [57]:
results_df_cat['Predicted Label'].value_counts()

기타소음      269
차량사이렌       4
이륜차주행음      1
Name: Predicted Label, dtype: int64

In [25]:
# '기타소음'의 개수와 비율 계산
total_files = len(results_df_cat)  # 전체 파일 개수
etc_noise_count = results_df_cat[results_df_cat['Predicted Label'] == '기타소음'].shape[0]  # '기타소음'으로 분류된 파일 개수
etc_noise_ratio = (etc_noise_count / total_files) * 100  # 비율 계산

# 결과 출력
print(f"Total Files: {total_files}")
print(f"'기타소음' Count: {etc_noise_count}")
print(f"'기타소음' Ratio: {etc_noise_ratio:.2f}%")

Total Files: 274
'기타소음' Count: 269
'기타소음' Ratio: 98.18%


### 항공기

In [22]:
folder_path = "/home/ubuntu/data/etc_noise_data_test_04/교통소음/3.항공기/6.비행기"
results = []

for filename in tqdm(os.listdir(folder_path)):
    if filename.endswith(".wav"):
        file_path = os.path.join(folder_path, filename)
        predicted_label = classify_audio(file_path)
        results.append({"File": filename, "Predicted Label": predicted_label})

# 5. 결과를 DataFrame으로 변환하고 CSV로 저장
results_df_airplane = pd.DataFrame(results)
results_df_airplane.to_csv("airplane.csv", index=False)
print("Classification completed. Results saved to 'airplane.csv'")

100%|██████████| 191/191 [00:59<00:00,  3.20it/s]

Classification completed. Results saved to 'airplane.csv'


In [58]:
results_df_airplane['Predicted Label'].value_counts()

기타소음      172
이륜차주행음     11
차량주행음       7
차량사이렌       1
Name: Predicted Label, dtype: int64

In [23]:
# '기타소음'의 개수와 비율 계산
total_files = len(results_df_airplane)  # 전체 파일 개수
etc_noise_count = results_df_airplane[results_df_airplane['Predicted Label'] == '기타소음'].shape[0]  # '기타소음'으로 분류된 파일 개수
etc_noise_ratio = (etc_noise_count / total_files) * 100  # 비율 계산

# 결과 출력
print(f"Total Files: {total_files}")
print(f"'기타소음' Count: {etc_noise_count}")
print(f"'기타소음' Ratio: {etc_noise_ratio:.2f}%")

Total Files: 191
'기타소음' Count: 172
'기타소음' Ratio: 90.05%


### 헬리콥터

In [27]:
folder_path = "/home/ubuntu/data/etc_noise_data_test_04/교통소음/3.항공기/7.헬리콥터"
results = []

for filename in tqdm(os.listdir(folder_path)):
    if filename.endswith(".wav"):
        file_path = os.path.join(folder_path, filename)
        predicted_label = classify_audio(file_path)
        results.append({"File": filename, "Predicted Label": predicted_label})

# 5. 결과를 DataFrame으로 변환하고 CSV로 저장
results_df_helicopter = pd.DataFrame(results)
results_df_helicopter.to_csv("helicopter.csv", index=False)
print("Classification completed. Results saved to 'helicopter.csv'")

100%|██████████| 435/435 [00:53<00:00,  8.12it/s]

Classification completed. Results saved to 'helicopter.csv'


In [59]:
results_df_helicopter['Predicted Label'].value_counts()

기타소음     434
차량사이렌      1
Name: Predicted Label, dtype: int64

In [28]:
# '기타소음'의 개수와 비율 계산
total_files = len(results_df_helicopter)  # 전체 파일 개수
etc_noise_count = results_df_helicopter[results_df_helicopter['Predicted Label'] == '기타소음'].shape[0]  # '기타소음'으로 분류된 파일 개수
etc_noise_ratio = (etc_noise_count / total_files) * 100  # 비율 계산

# 결과 출력
print(f"Total Files: {total_files}")
print(f"'기타소음' Count: {etc_noise_count}")
print(f"'기타소음' Ratio: {etc_noise_ratio:.2f}%")

Total Files: 435
'기타소음' Count: 434
'기타소음' Ratio: 99.77%


### 가전 청소기

In [30]:
folder_path = "/home/ubuntu/data/etc_noise_data_test_04/가전/12.청소기"
results = []

for filename in tqdm(os.listdir(folder_path)):
    if filename.endswith(".wav"):
        file_path = os.path.join(folder_path, filename)
        predicted_label = classify_audio(file_path)
        results.append({"File": filename, "Predicted Label": predicted_label})

# 5. 결과를 DataFrame으로 변환하고 CSV로 저장
results_df_vacuum = pd.DataFrame(results)
results_df_vacuum.to_csv("vacuum.csv", index=False)
print("Classification completed. Results saved to 'vacuum.csv'")

100%|██████████| 102/102 [00:48<00:00,  2.12it/s]

Classification completed. Results saved to 'vacuum.csv'


In [60]:
results_df_vacuum['Predicted Label'].value_counts()

기타소음      97
이륜차주행음     2
차량경적       2
이륜차경적      1
Name: Predicted Label, dtype: int64

In [31]:
# '기타소음'의 개수와 비율 계산
total_files = len(results_df_vacuum)  # 전체 파일 개수
etc_noise_count = results_df_vacuum[results_df_vacuum['Predicted Label'] == '기타소음'].shape[0]  # '기타소음'으로 분류된 파일 개수
etc_noise_ratio = (etc_noise_count / total_files) * 100  # 비율 계산

# 결과 출력
print(f"Total Files: {total_files}")
print(f"'기타소음' Count: {etc_noise_count}")
print(f"'기타소음' Ratio: {etc_noise_ratio:.2f}%")

Total Files: 102
'기타소음' Count: 97
'기타소음' Ratio: 95.10%


## 교통소음

### 차량 사이렌

In [11]:
folder_path = "/home/ubuntu/data/test_data/raw_data_test/1.Car/2.siren_of_car"
results = []

for filename in tqdm(os.listdir(folder_path)):
    if filename.endswith(".wav"):
        file_path = os.path.join(folder_path, filename)
        predicted_label = classify_audio(file_path)
        results.append({"File": filename, "Predicted Label": predicted_label})

# 5. 결과를 DataFrame으로 변환하고 CSV로 저장
results_df_car_siren = pd.DataFrame(results)
results_df_car_siren.to_csv("car_siren.csv", index=False)
print("Classification completed. Results saved to 'car_siren.csv'")

100%|██████████| 249/249 [00:31<00:00,  7.91it/s]

Classification completed. Results saved to 'car_siren.csv'


In [12]:
results_df_car_siren['Predicted Label'].value_counts()

차량사이렌     240
차량주행음       4
이륜차주행음      3
기타소음        1
이륜차경적       1
Name: Predicted Label, dtype: int64

In [13]:
# '기타소음'의 개수와 비율 계산
total_files = len(results_df_car_siren)  # 전체 파일 개수
etc_noise_count = results_df_car_siren[results_df_car_siren['Predicted Label'] == '차량사이렌'].shape[0]  # '기타소음'으로 분류된 파일 개수
etc_noise_ratio = (etc_noise_count / total_files) * 100  # 비율 계산

# 결과 출력
print(f"Total Files: {total_files}")
print(f"'차량사이렌' Count: {etc_noise_count}")
print(f"'차량사이렌' Ratio: {etc_noise_ratio:.2f}%")

Total Files: 249
'차량사이렌' Count: 240
'차량사이렌' Ratio: 96.39%


### 차량 경적

In [14]:
folder_path = "/home/ubuntu/data/raw_data/1.Car/1.horn_of_car"
results = []

for filename in tqdm(os.listdir(folder_path)):
    if filename.endswith(".wav"):
        file_path = os.path.join(folder_path, filename)
        predicted_label = classify_audio(file_path)
        results.append({"File": filename, "Predicted Label": predicted_label})

# 5. 결과를 DataFrame으로 변환하고 CSV로 저장
results_df_car_horn = pd.DataFrame(results)
results_df_car_horn.to_csv("car_horn.csv", index=False)
print("Classification completed. Results saved to 'car_horn.csv'")

100%|██████████| 3189/3189 [05:27<00:00,  9.73it/s]

Classification completed. Results saved to 'car_horn.csv'


In [15]:
results_df_car_horn['Predicted Label'].value_counts()

차량경적      3157
이륜차경적       16
이륜차주행음      11
차량사이렌        2
기타소음         2
차량주행음        1
Name: Predicted Label, dtype: int64

In [16]:
# '기타소음'의 개수와 비율 계산
total_files = len(results_df_car_horn)  # 전체 파일 개수
etc_noise_count = results_df_car_horn[results_df_car_horn['Predicted Label'] == '차량경적'].shape[0]  # '기타소음'으로 분류된 파일 개수
etc_noise_ratio = (etc_noise_count / total_files) * 100  # 비율 계산

# 결과 출력
print(f"Total Files: {total_files}")
print(f"'차량경적' Count: {etc_noise_count}")
print(f"'차량경적' Ratio: {etc_noise_ratio:.2f}%")

Total Files: 3189
'차량경적' Count: 3157
'차량경적' Ratio: 99.00%


### 차량 주행음

In [17]:
folder_path = "/home/ubuntu/data/test_data/raw_data_test/1.Car/3.driving_sound_of_car"
results = []

for filename in tqdm(os.listdir(folder_path)):
    if filename.endswith(".wav"):
        file_path = os.path.join(folder_path, filename)
        predicted_label = classify_audio(file_path)
        results.append({"File": filename, "Predicted Label": predicted_label})

# 5. 결과를 DataFrame으로 변환하고 CSV로 저장
results_df_car_driving = pd.DataFrame(results)
results_df_car_driving.to_csv("car_driving.csv", index=False)
print("Classification completed. Results saved to 'car_driving.csv'")

 32%|███▏      | 73/227 [00:09<00:21,  7.31it/s]


KeyboardInterrupt: 

In [ ]:
results_df_car_driving['Predicted Label'].value_counts()

In [ ]:
# '기타소음'의 개수와 비율 계산
total_files = len(results_df_car_driving)  # 전체 파일 개수
etc_noise_count = results_df_car_driving[results_df_car_driving['Predicted Label'] == '차량주행음'].shape[0]  # '기타소음'으로 분류된 파일 개수
etc_noise_ratio = (etc_noise_count / total_files) * 100  # 비율 계산

# 결과 출력
print(f"Total Files: {total_files}")
print(f"'차량주행음' Count: {etc_noise_count}")
print(f"'차량주행음' Ratio: {etc_noise_ratio:.2f}%")

### 이륜차 경적

In [ ]:
folder_path = "/home/ubuntu/data/test_data/raw_data_test/2.Motorcycle/4.horn_of_motorcycle"
results = []

for filename in tqdm(os.listdir(folder_path)):
    if filename.endswith(".wav"):
        file_path = os.path.join(folder_path, filename)
        predicted_label = classify_audio(file_path)
        results.append({"File": filename, "Predicted Label": predicted_label})

# 5. 결과를 DataFrame으로 변환하고 CSV로 저장
results_df_motorcycle_horn = pd.DataFrame(results)
results_df_motorcycle_horn.to_csv("motorcycle_horn.csv", index=False)
print("Classification completed. Results saved to 'motorcycle_horn.csv'")

In [ ]:
results_df_motorcycle_horn['Predicted Label'].value_counts()

In [ ]:
# '기타소음'의 개수와 비율 계산
total_files = len(results_df_motorcycle_horn)  # 전체 파일 개수
etc_noise_count = results_df_motorcycle_horn[results_df_motorcycle_horn['Predicted Label'] == '이륜차경적'].shape[0]  # '기타소음'으로 분류된 파일 개수
etc_noise_ratio = (etc_noise_count / total_files) * 100  # 비율 계산

# 결과 출력
print(f"Total Files: {total_files}")
print(f"'이륜차경적' Count: {etc_noise_count}")
print(f"'이륜차경적' Ratio: {etc_noise_ratio:.2f}%")

### 이륜차 주행음

In [ ]:
folder_path = "/home/ubuntu/data/test_data/raw_data_test/2.Motorcycle/5.driving_sound_of_motorcycle"
results = []

for filename in tqdm(os.listdir(folder_path)):
    if filename.endswith(".wav"):
        file_path = os.path.join(folder_path, filename)
        predicted_label = classify_audio(file_path)
        results.append({"File": filename, "Predicted Label": predicted_label})

# 5. 결과를 DataFrame으로 변환하고 CSV로 저장
results_df_motorcycle_driving = pd.DataFrame(results)
results_df_motorcycle_driving.to_csv("motorcycle_driving.csv", index=False)
print("Classification completed. Results saved to 'motorcycle_driving.csv'")

In [ ]:
results_df_motorcycle_driving['Predicted Label'].value_counts()

In [ ]:
# '기타소음'의 개수와 비율 계산
total_files = len(results_df_motorcycle_driving)  # 전체 파일 개수
etc_noise_count = results_df_motorcycle_driving[results_df_motorcycle_driving['Predicted Label'] == '이륜차주행음'].shape[0]  # '기타소음'으로 분류된 파일 개수
etc_noise_ratio = (etc_noise_count / total_files) * 100  # 비율 계산

# 결과 출력
print(f"Total Files: {total_files}")
print(f"'이륜차주행음' Count: {etc_noise_count}")
print(f"'이륜차주행음' Ratio: {etc_noise_ratio:.2f}%")